In [ ]:
pip install torch pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

In [ ]:
# Load the dataset
df = pd.read_csv("JoyVerseDataSet.csv")

# Extract labels and features
labels = df['Expression'].values
features = df.drop(columns=['Expression']).values

# Reshape to (samples, 468, 3)
features = features.reshape((features.shape[0], 468, 3))

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

In [ ]:
# Dataset class
class FaceMeshDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = FaceMeshDataset(X_train, y_train)
test_dataset = FaceMeshDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)


In [ ]:
class FaceMeshTransformer(nn.Module):
    def __init__(self, input_dim=3, model_dim=64, num_heads=4, num_layers=2, num_classes=5):
        super(FaceMeshTransformer, self).__init__()
        self.embedding = nn.Linear(input_dim, model_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(468 * model_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.embedding(x)  # (batch, seq_len, model_dim)
        x = x.permute(1, 0, 2)  # (seq_len, batch, model_dim)
        x = self.transformer_encoder(x)
        x = x.permute(1, 0, 2)  # back to (batch, seq_len, model_dim)
        return self.classifier(x)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FaceMeshTransformer(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

def train_model(model, loader, criterion, optimizer, epochs=100):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch_X, batch_y in loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

train_model(model, train_loader, criterion, optimizer)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1/100, Loss: 125.5695
Epoch 2/100, Loss: 33.4924
Epoch 3/100, Loss: 28.3439
Epoch 4/100, Loss: 28.5115
Epoch 5/100, Loss: 28.4597
Epoch 6/100, Loss: 28.4078
Epoch 7/100, Loss: 28.3571
Epoch 8/100, Loss: 28.3077
Epoch 9/100, Loss: 28.2748
Epoch 10/100, Loss: 28.2182
Epoch 11/100, Loss: 28.1606
Epoch 12/100, Loss: 28.1403
Epoch 13/100, Loss: 28.0957
Epoch 14/100, Loss: 28.0444
Epoch 15/100, Loss: 28.0060
Epoch 16/100, Loss: 27.9604
Epoch 17/100, Loss: 27.9287
Epoch 18/100, Loss: 27.9275
Epoch 19/100, Loss: 27.8643
Epoch 20/100, Loss: 27.8539
Epoch 21/100, Loss: 27.8077
Epoch 22/100, Loss: 27.7574
Epoch 23/100, Loss: 27.7396
Epoch 24/100, Loss: 27.7278
Epoch 25/100, Loss: 27.7135
Epoch 26/100, Loss: 27.6553
Epoch 27/100, Loss: 27.6338
Epoch 28/100, Loss: 27.6007
Epoch 29/100, Loss: 27.5971
Epoch 30/100, Loss: 27.5853
Epoch 31/100, Loss: 27.5393
Epoch 32/100, Loss: 27.5085
Epoch 33/100, Loss: 27.5139
Epoch 34/100, Loss: 27.4680
Epoch 35/100, Loss: 27.4470
Epoch 36/100, Loss: 27.4974


In [ ]:
emotion_classes = label_encoder.classes_
print("Emotion classes:", emotion_classes)

Emotion classes: ['Angry' 'Disgust' 'Fear' 'Happy' 'Neutral' 'Sad']


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch_X, batch_y in loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch_y.cpu().numpy())

    print("Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=emotion_classes))
    print("Accuracy:", accuracy_score(all_labels, all_preds))

evaluate_model(model, test_loader)

Classification Report:
              precision    recall  f1-score   support

       Angry       0.00      0.00      0.00        11
     Disgust       0.00      0.00      0.00         8
        Fear       0.00      0.00      0.00         1
       Happy       0.35      1.00      0.52        22
     Neutral       0.00      0.00      0.00        11
         Sad       0.00      0.00      0.00        10

    accuracy                           0.35        63
   macro avg       0.06      0.17      0.09        63
weighted avg       0.12      0.35      0.18        63

Accuracy: 0.3492063492063492


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Save the model
torch.save(model.state_dict(), "facemesh_transformer.pth")

# Save the label encoder classes
import json
with open("label_classes.json", "w") as f:
    json.dump(emotion_classes.tolist(), f)
